<a href="https://colab.research.google.com/github/FatemaSamir/pyspark/blob/main/Fatema_Samir_Streaming_P_S__Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#unZiping
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Spark_lab4/kospi.zip"
with ZipFile(file_name,'r') as zip:
 zip.extractall()
 print('Done')

Done


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 65 kB/s 
     |████████████████████████████████| 199 kB 47.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=36b89e541942f929bf26a1121158383fd6ca404870b3517f1c535edc420cd354
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [ ]:
# create Schema
from pyspark.sql.types import (StructType, StructField,
                               StringType, IntegerType, DateType, FloatType)

recordSchema = StructType([StructField('ID', IntegerType(), True),
                           StructField('Date', DateType(), True),
                           StructField('Open', FloatType(), True),
                           StructField('High', FloatType(), True),
                           StructField('Low', FloatType(), True),
                           StructField('Close', FloatType(), True),
                           StructField('Adj_Close', FloatType(), True),
                           StructField('Volume', IntegerType(), True)])

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [ ]:
df = spark.readStream.format('csv')\
.schema(recordSchema)\
.load('/content/drive/MyDrive/Spark_lab4/StreamInput/')

In [ ]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj_Close: float (nullable = true)
 |-- Volume: integer (nullable = true)



### Make sure the sataframe is streaming the files from the folder

In [ ]:
df.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:

In [ ]:
writer = df.writeStream.outputMode("append") \
    .format("memory")  \
    .queryName('stock')

### Start the write stream and make sure it works (read all columns from the table)

In [ ]:
query = writer.start()

In [ ]:
spark.sql('SELECT * FROM stock').show()

+---+----+----+----+---+-----+---------+------+
| ID|Date|Open|High|Low|Close|Adj_Close|Volume|
+---+----+----+----+---+-----+---------+------+
+---+----+----+----+---+-----+---------+------+



In [ ]:
spark.sql('SELECT * FROM stock').show()

+----+----------+-------+-------+-------+-------+---------+------+
|  ID|      Date|   Open|   High|    Low|  Close|Adj_Close|Volume|
+----+----------+-------+-------+-------+-------+---------+------+
|null|      null|   null|   null|   null|   null|     null|  null|
|   0|2000-01-04|22817.9|25696.8|22817.9|24879.3| 23510.88|108745|
|   1|2000-01-05|24523.9|26229.9|23670.9|24417.3|23074.295|175990|
|   2|2000-01-06|24381.7|24666.1|22746.8|22817.9|21562.865| 71746|
|   3|2000-01-07|22036.0|24879.3|22036.0|23884.2|22570.514|120984|
|   4|2000-01-10|24879.3|25519.1|23813.1|24061.9| 22738.44|151371|
|   5|2000-01-11|24168.5|25021.5|23955.2|24239.6|22906.365| 95943|
|   6|2000-01-12|24168.5|24452.8|23457.6|23670.9|22368.947| 61899|
|   7|2000-01-13|23670.9|24132.9|23102.2|23244.4|21965.906| 57538|
|   8|2000-01-14|23457.6|24168.5|22746.8|23244.4|21965.906| 84267|
|   9|2000-01-17|22533.6|23457.6|22533.6|23457.6|22167.377| 67807|
|  10|2000-01-18|23457.6|23742.0|22746.8|23422.1|22133.832| 27

In [ ]:
spark.sql('SELECT * FROM stock').show(100)

+----+----------+-------+-------+-------+-------+---------+------+
|  ID|      Date|   Open|   High|    Low|  Close|Adj_Close|Volume|
+----+----------+-------+-------+-------+-------+---------+------+
|null|      null|   null|   null|   null|   null|     null|  null|
|   0|2000-01-04|22817.9|25696.8|22817.9|24879.3| 23510.88|108745|
|   1|2000-01-05|24523.9|26229.9|23670.9|24417.3|23074.295|175990|
|   2|2000-01-06|24381.7|24666.1|22746.8|22817.9|21562.865| 71746|
|   3|2000-01-07|22036.0|24879.3|22036.0|23884.2|22570.514|120984|
|   4|2000-01-10|24879.3|25519.1|23813.1|24061.9| 22738.44|151371|
|   5|2000-01-11|24168.5|25021.5|23955.2|24239.6|22906.365| 95943|
|   6|2000-01-12|24168.5|24452.8|23457.6|23670.9|22368.947| 61899|
|   7|2000-01-13|23670.9|24132.9|23102.2|23244.4|21965.906| 57538|
|   8|2000-01-14|23457.6|24168.5|22746.8|23244.4|21965.906| 84267|
|   9|2000-01-17|22533.6|23457.6|22533.6|23457.6|22167.377| 67807|
|  10|2000-01-18|23457.6|23742.0|22746.8|23422.1|22133.832| 27

In [ ]:
df2 = spark.sql('SELECT * FROM stock')
df2.show(150)

+----+----------+-------+-------+-------+-------+---------+------+
|  ID|      Date|   Open|   High|    Low|  Close|Adj_Close|Volume|
+----+----------+-------+-------+-------+-------+---------+------+
|null|      null|   null|   null|   null|   null|     null|  null|
| 240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019|
| 241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791|
| 242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|
| 243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|
| 244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|
| 245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560|
| 246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|
| 247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|
| 248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|
| 249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|
| 250|2000-12-19|27722.7|28788.9|27651.6|27651.6|  26130.7|115

In [ ]:
# query.stop()

### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [ ]:
from pyspark.sql.functions import *
# drop na and add a new column "diff", which is the difference between high and low columns
df1 = df.dropna().withColumn("diff",df["High"] - df["Low"])

### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [ ]:
writer2 = df1.writeStream.outputMode("append") \
    .format("memory")  \
    .queryName('modified_data')

In [ ]:
query2 = writer2.start()

In [ ]:
spark.sql('SELECT * FROM modified_data').show()

+---+----+----+----+---+-----+---------+------+----+
| ID|Date|Open|High|Low|Close|Adj_Close|Volume|diff|
+---+----+----+----+---+-----+---------+------+----+
+---+----+----+----+---+-----+---------+------+----+



In [ ]:
df3 = spark.sql('SELECT * FROM modified_data')
df3.show(150)

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj_Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
|240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019| 995.2012|
|241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791| 710.8008|
|242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|533.10156|
|243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|1066.2988|
|244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|1208.4004|
|245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560| 639.7988|
|246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|1634.9004|
|247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|1492.7988|
|248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|1101.7988|
|249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|1066.1992|
|250|2000-12

### Write the generated data into files instead of the memory. 

In [ ]:
writerfiles = df1.writeStream.outputMode('append')\
.format('parquet')\
.option('path','/content/drive/MyDrive/Spark_lab4/OutputStream/')\
.option('checkpointLocation','/content/drive/MyDrive/Spark_lab4/OutputStream/chkpnt1').start()

### Stop the query. Now, try reading the generated parquet files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [ ]:

recordSchema2 = StructType([StructField('ID', IntegerType(), True),
                           StructField('Date', DateType(), True),
                           StructField('Open', FloatType(), True),
                           StructField('High', FloatType(), True),
                           StructField('Low', FloatType(), True),
                           StructField('Close', FloatType(), True),
                           StructField('Adj_Close', FloatType(), True),
                           StructField('Volume', IntegerType(), True),
                          StructField('diff', FloatType(), True)])
dfoutput = spark.read.format('parquet')\
.schema(recordSchema2)\
.load('/content/drive/MyDrive/Spark_lab4/OutputStream/part-00000-cc8a96ba-4167-4358-afb5-680608055b08-c000.snappy.parquet')

In [ ]:
dfoutput.show()

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj_Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
|240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019| 995.2012|
|241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791| 710.8008|
|242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|533.10156|
|243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|1066.2988|
|244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|1208.4004|
|245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560| 639.7988|
|246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|1634.9004|
|247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|1492.7988|
|248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|1101.7988|
|249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|1066.1992|
|250|2000-12

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|120|2000-06-20|22817.900391|23102.199219|21680.599609|22320.300781|21092.632813| 34466|1421.5996099999975|
|121|2000-06-21|21893.800781|22675.699219|21680.599609|22675.699219|21428.484375| 68651| 995.0996099999975|
|122|2000-06-22|23386.599609|23386.599609|     22462.5|23031.099609|21764.335938| 97209| 924.0996090000008|
|123|2000-06-23|22107.099609|24097.400391|22107.099609|     22889.0|21630.052734|199483|1990.3007819999984|
|124|2000-06-26|23102.199219|     24168.5|22569.099609|24026.300781|22704.796875|121969|1599.4003909999992|
|125|2000-06-27|24026.300781|25519.099609|     23742.0|24026.300781|22704.796875|113809|1777.0996090000008|
|126|2000-06-28|23884.199219

### Sort the dataframe based on the ID

In [ ]:
finalDFSorted = dfoutput.sort('ID')
finalDFSorted.show()

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj_Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
| 40|2000-02-29|25163.7|26087.7|24239.6|25519.1| 24115.49|233246|1848.0996|
| 41|2000-03-01|25519.1|25519.1|25519.1|25519.1| 24115.49|     0|      0.0|
| 42|2000-03-02|25767.9|29144.3|25767.9|28575.7|27003.973|408391|3376.4004|
| 43|2000-03-03|27793.8|29499.8|26798.6|27864.8|26332.176|216505|2701.2012|
| 44|2000-03-06|28291.3|29144.3|27367.3|28717.8|27138.256|170784|   1777.0|
| 45|2000-03-07|28575.7|29357.6|27935.9|28504.6|26936.781|109730|1421.6992|
| 46|2000-03-08|28078.1|28291.3|27154.0|28113.6| 26567.29| 89331|1137.3008|
| 47|2000-03-09|28717.8|29428.7|28078.1|28433.5|26869.592|196669|1350.5996|
| 48|2000-03-10|28646.8|28717.8|27367.3|28078.1| 26533.74| 87502|   1350.5|
| 49|2000-03-13|27154.0|28149.2|27154.0|27722.7| 26197.89| 59507| 995.1992|
| 50|2000-03